# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
#!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='Reacher_Linux/Reacher.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.24999999441206455


In [6]:
np.random.randn(num_agents, action_size)

array([[-0.06584041, -0.57694057, -1.44762865,  1.26717888]])

In [7]:
num_agents, action_size

(1, 4)

When finished, you can close the environment.

In [8]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Actor(nn.Module):

    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()

        self.dense = nn.Linear(state_size, 32)
        #self.drop = nn.Dropout(0.1)
        self.dense1 = nn.Linear(32, 16)
        #self.drop1 = nn.Dropout(0.1)
        self.dense2 = nn.Linear(16, action_size)
                
    def forward(self, x):
    
        x = F.relu(self.dense(x))
        #x = self.drop(x)
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        
        return F.tanh(x)
    
    
class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        
        #self.norm = nn.BatchNorm1d(state_size+action_size)
        self.dense_state = nn.Linear(state_size, 16)
        self.dense_state2 = nn.Linear(16, 32)
        self.dense_action = nn.Linear(action_size, 32)
        #self.drop = nn.Dropout(0.1)
        self.dense1 = nn.Linear(64, 32)
        #self.drop1 = nn.Dropout(0.1)
        self.dense2 = nn.Linear(32, 1)
       
    def forward(self, state, action):
        #x = self.norm(x)
        x = F.relu(self.dense_state(state))
        x = F.relu(self.dense_state2(x))
        x = torch.cat([x, F.relu(self.dense_action(action))], 1)
        #x = self.drop(x)
        x = F.relu(self.dense1(x))             
        value = F.relu(self.dense2(x))
        return value       


In [20]:
import torch.optim as optim
import random
from collections import namedtuple, deque

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#from previous project
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=min(self.batch_size, len(self.memory)))

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

## from: https://keras.io/examples/rl/ddpg_pendulum/    
class OUActionNoise:
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None):
        self.theta = theta
        self.mean = mean
        self.std_dev = std_deviation
        self.dt = dt
        self.x_initial = x_initial
        self.reset()

    def __call__(self):
        # Formula taken from https://www.wikipedia.org/wiki/Ornstein-Uhlenbeck_process.
        x = (
            self.x_prev
            + self.theta * (self.mean - self.x_prev) * self.dt
            + self.std_dev * np.sqrt(self.dt) * np.random.normal(size=self.mean.shape)
        )
        # Store x into x_prev
        # Makes next noise dependent on current one
        self.x_prev = x
        return x

    def reset(self):
        if self.x_initial is not None:
            self.x_prev = self.x_initial
        else:
            self.x_prev = np.zeros_like(self.mean)
    
    
class DDPG():
    def __init__(self, target_actor, actor, target_critic, critic, buffer, discount_rate):
        self.target_actor = target_actor
        self.target_critic = target_critic
        self.actor = actor
        self.critic = critic
        self.buffer = buffer
        self.discount_rate = discount_rate
        self.critic_optim = optim.Adam(self.critic.parameters(), lr=2e-3)
        self.actor_optim = optim.Adam(self.actor.parameters(), lr=1e-3)
    
    def learn(self, tau=0.01):
        if len(self.buffer.memory) > self.buffer.batch_size:
            states, actions, rewards, next_states, dones = self.buffer.sample()
            qnext = self.target_critic(next_states, self.target_actor(next_states))
            y = rewards + (1-dones)*self.discount_rate * qnext
            critic_loss = F.mse_loss(y, self.critic(states, actions))
            self.critic_optim.zero_grad()
            critic_loss.backward()
            self.critic_optim.step()

            actor_loss = -torch.mean(self.critic(states,self.actor(states)))
            self.actor_optim.zero_grad()
            actor_loss.backward()
            self.actor_optim.step()

            self.soft_update(self.actor, self.target_actor, tau)
            self.soft_update(self.critic, self.target_critic, tau)
    
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [21]:
discount_rate = 0.99

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
target_actor = Actor(state_size, action_size).to(device)
target_critic = Critic(state_size, action_size).to(device)

buffer = ReplayBuffer(action_size, 100000, 64, 0)
# noise = OUActionNoise(np.zeros(action_size), 0.01*np.ones(action_size), 
#                       x_initial=np.random.normal(action_size))
noise = OUActionNoise(np.zeros(action_size), 0.01*np.ones(action_size), theta=0.4,
                      dt = 0.05,
                      x_initial=np.random.normal(size=action_size))


agent = DDPG(target_actor, actor, target_critic, critic, buffer, discount_rate)

In [22]:
actor.dense.weight

Parameter containing:
tensor([[-0.0577, -0.1636, -0.1412,  ..., -0.0154,  0.1469, -0.0606],
        [-0.1333, -0.0632,  0.0838,  ..., -0.1403,  0.1541,  0.0295],
        [ 0.1065,  0.1461, -0.1613,  ..., -0.0142,  0.0156, -0.1339],
        ...,
        [ 0.1172,  0.1545, -0.1065,  ...,  0.1437,  0.1536,  0.0048],
        [ 0.0814,  0.0637, -0.0454,  ...,  0.1671, -0.0732, -0.0114],
        [ 0.1130, -0.0834, -0.0011,  ..., -0.1506, -0.0138, -0.1651]])

In [ ]:
nepisodes = 500

prev_actor_weight = actor.dense.weight.detach().numpy()
#prev_critic_weight = critic.dense.weight.detach().numpy()

avg_scores = []
for i in range(nepisodes):
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actor.eval()
        with torch.no_grad():
            actions = actor(torch.from_numpy(states).float().to(device)).cpu().numpy() + \
                    noise() # select an action (for each agent)
        actor.train()
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)      
        dones = env_info.local_done                        # see if episode finished
        buffer.add(states, actions, rewards, next_states, dones)
        
        agent.learn()
        
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            print("Episode %d: score %f" % (i, scores.mean()))
            avg_scores.append(scores.mean())
            if i % 10 == 0:
                print(actor.dense.weight)
                #prev_actor_weight = actor.dense.weight.detach().numpy()
                #pass
                #print(actor.dense.weight, critic.dense.weight)
            break

Episode 0: score 0.450000
Parameter containing:
tensor([[ 0.0709, -0.2038, -0.1745,  ...,  0.0142,  0.1469, -0.0902],
        [-0.1379,  0.0130,  0.1634,  ..., -0.0618,  0.1541, -0.0490],
        [ 0.0853,  0.1882, -0.1431,  ...,  0.0037,  0.0156, -0.1519],
        ...,
        [-0.1620,  0.1668,  0.0281,  ...,  0.1644,  0.1536, -0.0158],
        [ 0.1480,  0.0611, -0.0650,  ...,  0.1741, -0.0732, -0.0184],
        [ 0.2191, -0.1903,  0.0200,  ..., -0.0884, -0.0138, -0.2272]])
Episode 1: score 0.530000
Episode 2: score 0.550000
Episode 3: score 0.630000
Episode 4: score 1.270000
Episode 5: score 0.020000
Episode 6: score 0.480000
Episode 7: score 0.850000
Episode 8: score 1.040000
Episode 9: score 2.860000
Episode 10: score 2.930000
Parameter containing:
tensor([[-0.3463, -0.0660,  0.1189,  ..., -0.1516,  0.1469,  0.0662],
        [-0.1797, -0.0090,  0.2431,  ...,  0.1144,  0.1541, -0.3728],
        [ 0.2452, -0.0890, -0.2153,  ...,  0.2722,  0.0156, -0.8543],
        ...,
        [-0.

Episode 99: score 1.610000
Episode 100: score 1.310000
Parameter containing:
tensor([[-6.8243e-02, -1.2738e-01,  6.5580e-01,  ..., -2.2066e-01,
          1.4685e-01, -9.7351e-02],
        [-1.3713e-01,  8.0619e-02,  1.6584e-02,  ...,  4.0994e-01,
          1.5409e-01, -9.5287e-02],
        [ 1.4441e-01,  3.1444e-01, -5.2280e-01,  ...,  3.6492e-01,
          1.5649e-02,  9.1971e-01],
        ...,
        [ 4.2350e-01, -1.6813e-01,  9.3324e-01,  ..., -1.7950e-05,
          1.5359e-01, -2.7733e-01],
        [ 4.1352e-01, -2.7722e-01, -5.6813e-01,  ..., -3.3475e-01,
         -7.3200e-02,  2.1995e+00],
        [ 3.2184e-01, -7.4410e-03, -5.2140e-01,  ..., -2.0236e-02,
         -1.3758e-02,  1.4137e+00]])
Episode 101: score 0.000000
Episode 102: score 0.250000
Episode 103: score 0.160000
Episode 104: score 0.210000
Episode 105: score 1.290000
Episode 106: score 0.000000
Episode 107: score 0.250000
Episode 108: score 0.350000
Episode 109: score 0.000000
Episode 110: score 0.210000
Parameter c

Episode 206: score 0.660000
Episode 207: score 0.450000
Episode 208: score 0.350000
Episode 209: score 1.080000
Episode 210: score 0.350000
Parameter containing:
tensor([[-0.2019, -0.3011,  1.2074,  ..., -0.0818,  0.1469,  0.1105],
        [-0.0170,  0.0734,  0.1894,  ...,  0.6155,  0.1541,  0.1281],
        [ 0.2132,  0.4387, -0.3703,  ...,  0.4071,  0.0156,  0.9291],
        ...,
        [ 0.5286, -0.3121,  0.9289,  ...,  0.1262,  0.1536, -0.4568],
        [ 0.3988, -0.0411, -0.5864,  ..., -0.3465, -0.0732,  2.5800],
        [ 0.2529,  0.2519, -0.7912,  ...,  0.0148, -0.0138,  1.7201]])
Episode 211: score 0.560000
Episode 212: score 0.150000
Episode 213: score 0.140000
Episode 214: score 0.260000
Episode 215: score 0.000000
Episode 216: score 0.150000
Episode 217: score 0.460000
Episode 218: score 0.280000
Episode 219: score 0.520000
Episode 220: score 0.050000
Parameter containing:
tensor([[-0.2019, -0.3011,  1.2074,  ..., -0.0818,  0.1469,  0.1105],
        [-0.0170,  0.0734,  0.18

In [ ]:
actor(torch.from_numpy(states).float().to(device)).cpu().detach().numpy()

In [ ]:
x = noise()

In [ ]:
x

In [ ]:
actor(torch.from_numpy(states).float().to(device)).cpu().detach().numpy()+x